## MIDI Data Combination V5
(1). 기준
- tick
- sec 0.1 초 단위

(2). precheck
- track 길이 / track_name 동일
- 시간 길이 거의 비슷

### 피아노 Midi 사이트
- https://www.freepianotutorials.net/2023/12/ludwig-goransson-can-you-hear-music.html#more
- http://www.piano-midi.de/bach.htm

### Ⅰ. 라이브러리

In [157]:
import os
import editdistance
import numpy as np
import pandas as pd

import mido
from mido import MidiFile, MidiTrack, MetaMessage, Message
from tqdm.notebook import tqdm

In [158]:
data_folder = "midi_data/butterfly"

In [159]:
sec_base = True
seq_base = True

sec_unit  = 0.1

In [160]:
###################################################
############  트랙 기본 정보
###################################################

def print_midi_info(mid):
    
    print("파일 이름: ", mid.filename)
    print("총 재생 시간: ", mid.length)
    
    if mid.tracks:
        total_tracks = len(mid.tracks)
        
        print("-"*70)
        print("트랙 이름: ", mid.tracks[0].name)
        print("총 트랙의 수: ", total_tracks)
        
        for i in range(total_tracks):
            if mid.tracks[i].name:
                track_name = mid.tracks[i].name
            else:
                track_name = None
            
            print(f"[{i+1}]. {mid.tracks[i].name}")
        
    else:
        print("트랙이 없습니다")

In [161]:
def load_midi_data(input_name, target_name):
    input_path  = os.path.join(data_folder, input_name)
    target_path = os.path.join(data_folder, target_name)
    
    input_mid  = mido.MidiFile(input_path)
    target_mid = mido.MidiFile(target_path)
    
    print("="*20, "[Input Midi Data]", "="*20)
    print_midi_info(input_mid)
    
    print()
    
    print("="*20, "[Target Midi Data]", "="*20)
    print_midi_info(target_mid)
    
    return input_mid, target_mid

### Ⅱ. 데이터 추출하기

In [6]:
def load_mid_Info_from_csv(input_mid, target_mid):
    
    input_name  = input_mid.filename
    target_name = target_mid.filename
    
    input_csv  = input_name.split(".")[0]+".csv"
    target_csv = target_name.split(".")[0]+".csv"
    
    input_info = pd.read_csv(input_csv)
    target_info = pd.read_csv(target_csv)
    
    #input_info = pd.read_csv(input_csv, index_col = 0)
    #target_info = pd.read_csv(target_csv, index_col = 0)
    
    return input_info, target_info


#### [1]. PreCheck
- track 길이
- track 이름
- 음의 길이

In [122]:
def pre_check(input_mid, target_mid):
    input_track_len  = len(input_mid.tracks)
    target_track_len = len(target_mid.tracks)
    
    # track 길이 비교
    if input_track_len != target_track_len:
        print(f"input track len({input_track_len}) != target track len({target_track_len})")
        return False
    
    # track_name 비교
    input_track_name  = [ msg.name for msg in input_mid if msg.type == 'track_name']
    target_track_name = [ msg.name for msg in target_mid if msg.type == 'track_name']
    
    if input_track_name != target_track_name:
        print(f"input track name({input_track_name}) != target track name({target_track_name})")
        return False
    
    # track 시간 비교 ( time_diff_interval 이상 비교 불가)
    input_mid_time  = input_mid.length
    target_mid_time = target_mid.length
    
    time_diff_interval = 20
    
    if abs(input_mid_time - target_mid_time) > time_diff_interval:
        print(f"input time({input_mid_time}) != target time({target_mid_time})")
        return False
    
    return True


#### [2]. MetaMessage

#### [3]. Message
- mean_velocity ( 평균 세기 ) : (( 셈여림 ))에 활용
- diff_velocity ( 세기 변화 ) 
    - 현재 노트의 평균 세기 - 이전 노트의 평균 세기
    - 음표의 세기, 빠르기의 변화에 활용
- dynamic ( 셈여림 )
|셈여림(dynamic)|정의|평균 velocity 범위|
|:--:|:--:|:--:|
|ppp(피아니시시모)|아주 여리게|0~35|
|pp(피아니시모)|매우 여리게|36~48|
|p(피아노)|여리게|49~61|
|mp(메조 피아노)|조금 여리게|62~74|
|mf(메조 포르테)|조금 세게|75~87|
|f(포르테)|세게|88~100|
|ff(포르티시모)|매우 세게|101~113|
|fff(포르티시시모)|아주 세게|114~127|

In [110]:
def create_df_info():
    #df_col = ['sec',
    #          'msg_type', 'channel', 'note', 'velocity', 'dynamic', 'accent', 'count',
    #          'main_vol','depth', 'pedal', 'pan', 'tempo']
    
    df_col = ['sec',
              'msg_type', 'channel', 'note', 'velocity', 'dynamic', 'accent', 'count',
              'main_vol','depth', 'pedal', 'pan']
    df = pd.DataFrame(columns = df_col)
    
    df_tempo_col = ['tempo', 'tick']
    df_tempo = pd.DataFrame(columns = df_tempo_col)

    df_info = { MetaMessage:{}, Message:df }
    
    return df_info, df_tempo

def info_to_list(cur_info):
    
    #cur_temp = [cur_info['sec'],
    #            cur_info['msg_type'], cur_info['channel'], cur_info['note'], 
    #            cur_info['velocity'], cur_info['dynamic'], cur_info['accent'], cur_info['count'],
    #            cur_info['main_vol'], cur_info['depth'], cur_info['pedal'], cur_info['pan'], cur_info['tempo']]
    
    cur_temp = [cur_info['sec'],
                cur_info['msg_type'], cur_info['channel'], cur_info['note'], 
                cur_info['velocity'], cur_info['dynamic'], cur_info['accent'], cur_info['count'],
                cur_info['main_vol'], cur_info['depth'], cur_info['pedal'], cur_info['pan']]
    return cur_temp
                

def initialize_cur_info():
    # cur_info = {'sec':0,
    #            'msg_type': [], 'channel':[], 'note':[],  'velocity':[], 'dynamic':"", 'accent':0, 'count':0,
    #            'main_vol':-1, 'depth':-1, 'pedal':-1, 'pan':-1, 'tempo':[] }
    
    cur_info = {'sec':0,
                'msg_type': [], 'channel':[], 'note':[],  'velocity':[], 'dynamic':"", 'accent':0, 'count':0,
                'main_vol':-1, 'depth':-1, 'pedal':-1, 'pan':-1}
    return cur_info

def check_not_list(msg_list):
    if type(msg_list) == list:
        return msg_list
    else:
        msg_list = [msg_list]
        return msg_list

In [9]:
def tempo_info_list(mid_info, tempo_info):
    
    #default tempo setting
    if 'set_tempo' not in mid_info[MetaMessage]:
        msg = mido.MetaMessage('set_tempo', tempo = 500000)
        
        mid_info[MetaMessage][msg.type] = [msg]
    
    tempo_list = mid_info[MetaMessage]['set_tempo']
    
    tempo_tick = 0
    
    for idx, msg in enumerate(tempo_list):
        tempo_tick += msg.time
            
        if idx==0:
            tempo_info.loc[idx,'tempo'] = msg.tempo
        else:
            tempo_info.loc[idx, 'tempo'] = msg.tempo
            tempo_info.loc[idx-1, 'tick'] = tempo_tick
            
    tempo_info.loc[tempo_info.shape[0] - 1, 'tick'] = -1
                    
    return tempo_info

def find_tempo(tick, tempo_info):
    tempo_idx = 0
    last_tempo_idx = tempo_info.shape[0]
    
    cur_tempo = -1
    
    for tempo_idx in range(last_tempo_idx):
        if tempo_info.loc[tempo_idx, 'tick'] > tick:
            cur_tempo = tempo_info.loc[tempo_idx, 'tempo']
            break
            
    if cur_tempo == -1:
        cur_tempo = tempo_info.loc[last_tempo_idx - 1, 'tempo']
        
    return cur_tempo

In [10]:
###################################################
############  셈여림 정보
###################################################
def distribute_dynamic(mean_velocity):
    dynamic = ''
    if 0 <= mean_velocity < 36:
        dynamic = 'ppp'
    elif 36 <= mean_velocity < 49:
        dynamic = 'pp'
    elif 49 <= mean_velocity < 62:
        dynamic = 'p'
    elif 62 <= mean_velocity < 75:
        dynamic = 'mp'
    elif 75 <= mean_velocity < 88:
        dynamic = 'mf'
    elif 88 <= mean_velocity < 101:
        dynamic = 'f'
    elif 101 <= mean_velocity < 114:
        dynamic = 'ff'
    elif 114 <= mean_velocity <= 127:
        dynamic = 'fff'
    
    return dynamic

###################################################
############  셈여림, 악센트 계산 함수
###################################################
def calculate_dynamic_accent(cur_info, msg_df):
    
    last_idx = msg_df.shape[0]
    
    if last_idx>0:
        prev_velo = msg_df.loc[last_idx-1, 'velocity']
        prev_avg_velo = sum(prev_velo) / len(prev_velo) if prev_velo else 0
    else:
        prev_avg_velo = 0
        
    if cur_info['count'] > 0 :
        mean_velocity = sum(cur_info['velocity']) / len(cur_info['velocity'])
        diff_velocity = mean_velocity - prev_avg_velo
    else:
        mean_velocity = 0
        diff_velocity = 0
        
    dynamic = distribute_dynamic(mean_velocity)
    accent = 1 if 76 <= mean_velocity <= 127 and mean_velocity > 1.2 * prev_avg_velo else 0
    
    return dynamic, accent
    

In [11]:
def control_type(msg):
    
    ctl_type = None
    
    if msg.control == 1:
        ctl_type = 'modulation'
        
    elif msg.control == 7:
        ctl_type = 'main_vol'
        
    elif msg.control == 10:
        ctl_type = 'pan'
        
    elif msg.control == 64:
        ctl_type = 'pedal'
        
    elif msg.control >= 91 and msg.control <= 93:
        ctl_type = 'depth'
        
    return ctl_type

In [90]:
def process_msg(msg, info):
    
    if msg.type == 'note_on' or msg.type == 'note_off':
        
        if msg.type == 'note_on' and msg.velocity == 0:
            msg_type = 'note_off'
        else:
            msg_type = msg.type
                    
        info['msg_type'].append(msg_type)
        info['channel'].append(msg.channel)
        info['note'].append(msg.note)
        info['velocity'].append(msg.velocity)
                    
        info['count']+=1
        
    elif msg.type == 'program_change':
        print("program_change")
        return False
        
    elif msg.type == 'control_change':
        
        ctl_type = control_type(msg)
        
        # 수집하지 않은 control_type
        if ctl_type == None:
            return False
        
        if info[ctl_type] == -1:
            info[ctl_type] = msg.value
        else:
            info[ctl_type] = check_not_list(info[ctl_type])
            info[ctl_type].append(msg.value)
    
    return True

In [108]:
def remain_check(info):
    
    if info['count'] > 0:
        return True
    
    if info['main_vol'] != -1:
        return True
    
    if info['depth'] != -1:
        return True
    
    if info['pedal'] != -1:
        return True
    
    if info['pan'] != -1:
        return True
    
    return False

In [118]:
def extract_mid_data(mid):
    
    mid_info, tempo_info = create_df_info()
    tick_base = True
    
    for idx, track in enumerate(mid.tracks):
        
        cur_tick = 0
        cur_sec  = 0
        cur_info = initialize_cur_info()
        
        
        for i, msg in tqdm(enumerate(track)):
            
            # MetaMessage
            if isinstance(msg, MetaMessage):
                                
                if msg.type not in mid_info[MetaMessage]:
                    mid_info[MetaMessage][msg.type] = []
                    
                mid_info[MetaMessage][msg.type].append(msg)
            
            elif isinstance(msg, Message):
                
                cur_tick = cur_tick + msg.time
                
                msg_tempo = find_tempo(cur_tick, tempo_info)
                msg_sec = mido.tick2second( cur_tick, mid.ticks_per_beat, msg_tempo )
                msg_sec = np.trunc(msg_sec*10) / 10 # 소수 첫째 자리까지 버림
                
                if msg_sec > cur_sec:                        
                    
                    dynamic, accent = calculate_dynamic_accent(cur_info, mid_info[Message])
                    
                    cur_info['dynamic'] = dynamic
                    cur_info['accent'] = accent
                    
                    cur_info['sec'] = cur_sec
                    
                    cur_temp = info_to_list(cur_info)
                    
                    info_cur = int(msg_sec * 10)
                    info_last = mid_info[Message].shape[0]
                                        
                    mid_info[Message].loc[info_last] = cur_temp
                    
                    if sec_base == True:
                        
                        # empty music
                        temp_info = initialize_cur_info()
                        temp_temp = info_to_list(temp_info)
                        
                        for sec in range(info_last + 1, info_cur):
                            temp_temp[0] = float(sec / 10)
                            
                            mid_info[Message].loc[sec] = temp_temp
                                                    
                    cur_info = initialize_cur_info()

                isProcess = process_msg(msg, cur_info)
                if isProcess == False:
                    continue
                
                # tempo
                # msg_tempo = find_tempo(cur_tick, tempo_info)
                # cur_info['tempo'].append(msg_tempo)
                
                cur_sec = msg_sec
            
        if idx == 1 and remain_check(cur_info) == True:

            # 잔여 cur_info 추가
            dynamic, accent = calculate_dynamic_accent(cur_info, mid_info[Message])

            cur_info['dynamic'] = dynamic
            cur_info['accent'] = accent

            cur_info['sec'] = cur_sec

            cur_temp = info_to_list(cur_info)

            info_cur = int(msg_sec * 10)
            info_last = mid_info[Message].shape[0]
            
            if sec_base == True:
                
                # empty music
                temp_info = initialize_cur_info()
                temp_temp = info_to_list(temp_info)
                        
                for sec in range(info_last + 1, info_cur):
                    temp_temp[0] = float(sec / 10)
                            
                    mid_info[Message].loc[sec] = temp_temp
                                                    
            cur_info = initialize_cur_info()
            
        # tempo setting
        if idx == 0:
            tempo_info_list(mid_info, tempo_info)
            
            
    mid_info[Message].replace(-1, 0, inplace = True)
    
    csv_name = mid.filename.split(".")[0]+".csv"
    mid_info[Message].to_csv(csv_name, index = False)
    
    return mid_info


In [114]:
def extrack_mid_info(input_mid, target_mid):
    
    input_info  = extract_mid_data(input_mid)
    target_info = extract_mid_data(target_mid)
    
    print("="*20, "  [Input_Info[Message]]  ", "="*20)
    display(input_info[Message])
    
    print()
    
    print("="*20, "  [target_Info[Message]]  ", "="*20)
    display(target_info[Message])
    
    return input_info, target_info

### Ⅲ. 실행

#### [1]. 실행 코드

In [120]:
# input_name = "river_flows_in_you_test_v2.mid"
# target_name = "river_flows_in_you_origin.mid"

input_name  = "butterfly_input_data.mid"
target_name = "butterfly_target_data.mid"

input_mid, target_mid = load_midi_data(input_name, target_name)

==================== [Input Midi Data] ====================
파일 이름:  midi_data/butterfly\butterfly_input_data.mid
총 재생 시간:  27.00000000000003
----------------------------------------------------------------------
트랙 이름:  butterfly
총 트랙의 수:  2
[1]. butterfly
[2]. PIANO

==================== [Target Midi Data] ====================
파일 이름:  midi_data/butterfly\butterfly_target_data.mid
총 재생 시간:  27.0
----------------------------------------------------------------------
트랙 이름:  butterfly
총 트랙의 수:  2
[1]. butterfly
[2]. PIANO


In [123]:
if pre_check(input_mid, target_mid):
    print("===== pre_check success =====")
    input_info, target_info = extrack_mid_info(input_mid, target_mid)
else:
    print("===== pre_check failed =====")

===== pre_check success =====


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

====================   [Input_Info[Message]]   ====================


,sec,msg_type,channel,note,velocity,dynamic,accent,count,main_vol,depth,pedal,pan
0,0.0,[],[],[],[],ppp,0,0,100,0,0,64
1,0.1,[],[],[],[],,0,0,0,0,0,0
2,0.2,[],[],[],[],,0,0,0,0,0,0
3,0.3,[],[],[],[],,0,0,0,0,0,0
4,0.4,[],[],[],[],,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
265,26.5,[],[],[],[],,0,0,0,0,0,0
266,26.6,[],[],[],[],,0,0,0,0,0,0
267,26.7,[],[],[],[],,0,0,0,0,0,0
268,26.8,[],[],[],[],,0,0,0,0,0,0



====================   [target_Info[Message]]   ====================


,sec,msg_type,channel,note,velocity,dynamic,accent,count,main_vol,depth,pedal,pan
0,0.0,[],[],[],[],ppp,0,0,100,0,0,64
1,0.1,[],[],[],[],,0,0,0,0,0,0
2,0.2,[],[],[],[],,0,0,0,0,0,0
3,0.3,[],[],[],[],,0,0,0,0,0,0
4,0.4,[],[],[],[],,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
265,26.5,[],[],[],[],,0,0,0,0,0,0
266,26.6,[],[],[],[],,0,0,0,0,0,0
267,26.7,[],[],[],[],,0,0,0,0,0,0
268,26.8,[],[],[],[],,0,0,0,0,0,0


In [364]:
if pre_check(input_mid, target_mid):
    print("===== pre_check success =====")
    input_info, target_info = load_mid_Info_from_csv(input_mid, target_mid)
else:
    print("===== pre_check failed =====")

#### [2]. 임시방편

In [137]:
input_msg_info  = pd.read_csv(data_folder+"/butterfly_input_data.csv",  keep_default_na=False)
target_msg_info = pd.read_csv(data_folder+"/butterfly_target_data.csv", keep_default_na=False)

In [138]:
display(input_msg_info)
print("="*25)
display(target_msg_info)

,sec,msg_type,channel,note,velocity,dynamic,accent,count,main_vol,depth,pedal,pan
0,0.0,[],[],[],[],ppp,0,0,100,0,0,64
1,0.1,[],[],[],[],,0,0,0,0,0,0
2,0.2,[],[],[],[],,0,0,0,0,0,0
3,0.3,[],[],[],[],,0,0,0,0,0,0
4,0.4,[],[],[],[],,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
265,26.5,[],[],[],[],,0,0,0,0,0,0
266,26.6,[],[],[],[],,0,0,0,0,0,0
267,26.7,[],[],[],[],,0,0,0,0,0,0
268,26.8,[],[],[],[],,0,0,0,0,0,0


,sec,msg_type,channel,note,velocity,dynamic,accent,count,main_vol,depth,pedal,pan
0,0.0,[],[],[],[],ppp,0,0,100,0,0,64
1,0.1,[],[],[],[],,0,0,0,0,0,0
2,0.2,[],[],[],[],,0,0,0,0,0,0
3,0.3,[],[],[],[],,0,0,0,0,0,0
4,0.4,[],[],[],[],,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
265,26.5,[],[],[],[],,0,0,0,0,0,0
266,26.6,[],[],[],[],,0,0,0,0,0,0
267,26.7,[],[],[],[],,0,0,0,0,0,0
268,26.8,[],[],[],[],,0,0,0,0,0,0


### Ⅳ. 비교 알고리즘

#### [0]. 사전 작업
- 정답 데이터의 tick 행 길이에 맞추는 작업
- 입력 데이터의 tick 행 길이가 적을 경우, 0 으로 처리

In [143]:
def align_and_save_dataframes(input_mid, target_mid):
    
    input_filename  = input_mid.filename
    target_filename = target_mid.filename
    
    input_file  = input_filename.split(".")[0] +".csv"
    target_file = target_filename.split(".")[0]+".csv"
    
    df_input  = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)
    
    print("Input shape: ", df_input.shape)
    print("Target shape:", df_target.shape)

    # target_file의 행 길이로 맞추기
    min_length = min(len(df_input), len(df_target))
    df_input_aligned = df_input.head(min_length)

    # target_file보다 행이 작으면 나머지 행은 0으로 채우기
    if len(df_input) < len(df_target):
        df_input_aligned = pd.concat([df_input_aligned, pd.DataFrame(0, index=range(len(df_target) - len(df_input)), columns=df_input.columns)])

    # 모든 컬럼에 대해 0으로 채우기
    for col in df_input.columns:
        if col not in df_target.columns:
            df_input_aligned[col] = 0

    # msg_type, Notes, Velocities 컬럼에 대해 빈 리스트 []를 NaN으로 처리
    for col in ['msg_type', 'channel', 'note', 'velocity']:
        df_input_aligned[col] = df_input_aligned[col].apply(lambda x: [] if x == 0 else x)

    df_input_aligned.to_csv(input_file, index=False)
    df_target.to_csv(target_file, index=False)
    
    return df_input_aligned, df_target

In [144]:
input_msg_info, target_msg_info = align_and_save_dataframes(input_mid, target_mid)

Input shape:  (270, 12)
Target shape: (270, 12)


#### [1]. 노트 정확도 ( 음정 정확도 : pitch accuracy )
- 입력 데이터와 정답 데이터의 Notes 를 비교하여, 현재 노트가 일치하면 +2 점을 부여
- [의문점]
    - 실수로 잘못 쳐서 한번 밀려서 연주한 경우 / 계속 밀리는 문제 ( -1, +1 을 해야하는 가 )

In [156]:
def calculate_note_accuracy(df_input, df_target):

    total_accuracy = 0

    for index in range(len(df_input)):
        notes_input = eval(df_input.at[index, 'note'])
        notes_target = eval(df_target.at[index, 'note'])

        # 0으로 된 값 리스트로 처리
        if isinstance(notes_input, int):
            notes_input = [notes_input]
        if isinstance(notes_target, int):
            notes_target = [notes_target]
        
        row_accuracy = 0

        for note_input in set(notes_input):
            for note_target in set(notes_target):
                if note_input == note_target:
                    row_accuracy += 1
                    #print('-'*20)
                    #print(f'입력 데이터: {notes_input}')
                    #print(f'정답 데이터: {notes_target}')
                    #print(f'!!!{row_accuracy}점 추가!!!')
                    #print('-'*20)

                #print(f'비교 데이터 - 입력 데이터: {note_input} / 정답 데이터: {note_target}')
                pass

        total_accuracy += row_accuracy
        #print(f'현재 점수: {total_accuracy}점')
        #print()
        
    print(df_input)
    print(">>>>>>>>>>>>>. ", len(df_input), ",,,,,", max(len(set(notes_input)), len(set(notes_target))))

    # 전체 정확도 계산
    max_possible_accuracy = len(df_input)
    accuracy_percentage = min(100, (total_accuracy / max_possible_accuracy) * 100)
    
    accuracy_percentage_round = np.round(accuracy_percentage)*10000 / 10000

    return f'{accuracy_percentage_round}%'

accuracy = calculate_note_accuracy(input_msg_info, target_msg_info)
print(f'음정 정확도: {accuracy}')

      sec msg_type channel note velocity dynamic  accent  count  main_vol  \
0     0.0       []      []   []       []     ppp       0      0       100   
1     0.1       []      []   []       []     NaN       0      0         0   
2     0.2       []      []   []       []     NaN       0      0         0   
3     0.3       []      []   []       []     NaN       0      0         0   
4     0.4       []      []   []       []     NaN       0      0         0   
..    ...      ...     ...  ...      ...     ...     ...    ...       ...   
265  26.5       []      []   []       []     NaN       0      0         0   
266  26.6       []      []   []       []     NaN       0      0         0   
267  26.7       []      []   []       []     NaN       0      0         0   
268  26.8       []      []   []       []     NaN       0      0         0   
269  26.9       []      []   []       []     NaN       0      0         0   

     depth pedal  pan  
0        0     0   64  
1        0     0    0  
2  

In [153]:
accuracy = calculate_note_accuracy(input_msg_info, target_msg_info)
print(f'음정 정확도: {accuracy}')

>>>>>>>>>>>>>.  270 ,,,,, 0
음정 정확도: 70.0%


#### [2]. 셈여림
- 코드 작업이 더 필요한 것처럼 보임

In [425]:
def calculate_dynamic_level(input_df, target_df):

    input_counts  = input_df['dynamic'].value_counts().to_dict()
    target_counts = target_df['dynamic'].value_counts().to_dict()

    return input_counts, target_counts



In [426]:
# 셈여림
dynamic_note_score1 = calculate_dynamic_level(input_msg_info, target_msg_info)
print(dynamic_note_score1)

({'ppp': 27909, 'p': 7970, 'pp': 5529, 'mp': 3386, 'mf': 8}, {'pp': 29536, 'ppp': 14050, 'p': 1088, 'mp': 128})


In [ ]:
print(len(input_mid.tracks[1]))
print("======================")
print(len(target_mid.tracks[1]))


print(input_mid.ticks_per_beat)
print("======================")
print(target_mid.ticks_per_beat)


input_time = 0
for msg in input_mid.tracks[1]:
    if isinstance(msg, Message):
        if msg.type == "note_on" or msg.type == "note_off":
            input_time+=msg.time
        
print(input_time)

target_time = 0
for msg in target_mid.tracks[1]:
    if isinstance(msg, Message):
        if msg.type == "note_on" or msg.type == "note_off":
            target_time+=msg.time
        
print(target_time)

In [ ]:
display(input_info)
print("================")
display(target_info)